# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [ ]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("MAPBOX_ACCESS_TOKEN")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [ ]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [ ]:
# Getting the data from the top 10 expensive neighbourhoods

average_house_values_neighbourhood = to_data.groupby('neighbourhood')[['average_house_value']].mean()
df_house_values_neighbourhood = average_house_values_neighbourhood.sort_values(by='average_house_value',ascending=False).head(10)
df_house_values_neighbourhood = df_house_values_neighbourhood.reset_index()

# Calculate the mean number of dwelling types units per year

dwelling_types_year_neighbourhood = to_data.groupby(['year','neighbourhood'])[['single_detached_house',
                                    'apartment_five_storeys_plus',
                                    'movable_dwelling',
                                    'semi_detached_house',
                                    'row_house',
                                    'duplex',
                                    'apartment_five_storeys_less',
                                    'other_house']].mean()

# House values per neighbourhood per year
avg_house_values_neighbourhood = to_data.groupby(['year','neighbourhood'])[['average_house_value']].mean()

# Calculate the average monthly shelter costs for owned and rented dwellings

avg_shelter_costs = to_data.groupby('year')[['shelter_costs_owned','shelter_costs_rented']].mean()

#Sum number of dwelling types units per year

dwelling_types_year = to_data.groupby('year')[['single_detached_house',
                                    'apartment_five_storeys_plus',
                                    'movable_dwelling',
                                    'semi_detached_house',
                                    'row_house',
                                    'duplex',
                                    'apartment_five_storeys_less',
                                    'other_house']].sum()
 
 # Avg House Values
average_house_values = to_data.groupby('year')[['average_house_value']].mean()

### Panel Visualization Functions

In [ ]:
# Define Panel visualization functions
def neighbourhood_map():
    """Neighbourhood Map"""
   
    map_df = pd.concat([average_house_values_neighbourhood,df_neighbourhood_locations], axis=1, join="outer")
    map_df = map_df.reset_index()
    return px.scatter_mapbox(
            map_df,
            lat="lat",
            lon="lon",
            color="average_house_value"
            )

def create_bar_chart(data, title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    
    return data.hvplot.bar(title=title,xlabel=xlabel,ylabel=ylabel,color=color)

def create_line_chart(data, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    
    return data.hvplot.line(title=title,xlabel=xlabel,ylabel=ylabel,color=color)

def average_house_value():
    """Average house values per year."""
    
    return create_line_chart(
            average_house_values['average_house_value'],
            "Average House Values in Toronto",
            'Year',
            'Avg House Values',
            'cyan'
        )

def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    
    return average_house_values_neighbourhood.hvplot.line(
                x = 'neighbourhood',
                y = 'average_house_value',
                rot = 60,
                height = 500,
                title = 'Average house values by neighbourhood',
                xlabel = 'Neighbourhood',
                ylabel = 'Avg House Value'
            )

def number_dwelling_types():
    """Number of dwelling types per year"""
    
    colour = ['tomato','green','blue','yellow','orange','magenta','cyan','limegreen']

    return dwelling_types_year_neighbourhood.hvplot.bar(
                x='year',
                groupby='neighbourhood',
                height=400,
                rot=60,
                color=colour
                )

def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""
    
    return avg_house_values_neighbourhood.hvplot.line( x='year',y='average_house_value',groupby='neighbourhood', height=400, width=600)


def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    
    return df_house_values_neighbourhood.hvplot.bar(
                x = 'neighbourhood',
                y = 'average_house_value',
                rot = 60,
                height = 500,
                title = 'Top 10 Most Expensive Neighbourhoods',
                xlabel = 'Neighbourhood',
                ylabel = 'Avg House Value'
            )

def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    df = avg_house_values_neighbourhood.reset_index()
    return px.sunburst(df,path= ['neighbourhood','year'], values='average_house_value', color='year')

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [81]:
# Create a Title for the Dashboard
title = pn.Column("# Reeal Estate analysis of Toronto from 2001 to 2016\n")

# Define a welcome text
desc = pn.Column("### This dashboard details about the real estate scene in Toronto city during 2001 and 2016.")

# Create a tab layout for the dashboard
# Neighbourhood map
welcome = pn.Column( neighbourhood_map())

#Yearly Market Analysis
market_year = pn.Column(
    create_bar_chart(dwelling_types_year.iloc[0],
                'Dwelling Types in Toronto in 2001',
                '2001',
                'Dwelling Types Units',
                'tomato'
                )+
    create_bar_chart(dwelling_types_year.iloc[1],
                'Dwelling Types in Toronto in 2006',
                '2006',
                'Dwelling Types Units',
                'blue'
                )+
    create_bar_chart(dwelling_types_year.iloc[2],
                'Dwelling Types in Toronto in 2011',
                '2011',
                'Dwelling Types Units',
                'green'
                )+
    create_bar_chart(dwelling_types_year.iloc[3],
                'Dwelling Types in Toronto in 2016',
                '2016',
                'Dwelling Types Units',
                'orange'
                )
)

# Shelter Costs Vs House Values

shelter_costs = pn.Column(
    create_line_chart(
        avg_shelter_costs['shelter_costs_owned'],
        "Average Monthly Shelter Cost for Owned Dwellings in Toronto",
        'Year',
        'Avg Monthly Shelter Costs',
        'blue'
    )+
    create_line_chart(
        avg_shelter_costs['shelter_costs_rented'],
        "Average Monthly Shelter Cost for Rented Dwellings in Toronto",
        'Year',
        'Avg Monthly Shelter Costs',
        'magenta'
    )+
    average_house_value()
)

# Neighbourhood Analysis
neighbourhood_analysis = pn.Column( average_house_value_snapshot, number_dwelling_types)

# Top Expensive Neighbourhoods
expensive = pn.Row(top_most_expensive_neighbourhoods,sunburts_cost_analysis)
# Create the main dashboard
Toronto_realty_dashboard = pn.Tabs(
    (
        "Welcome",
        welcome
    ),
    (
        "Yearly Market Analysis",
        market_year
    ),
    (
        "Shelter Costs Vs House Values",
        shelter_costs
    ),
    (
        "Neighbourhood Analysis",
        neighbourhood_analysis
    ),
    (
        "Top Expensive Neighbourhoods",
        expensive
    )
)

dashboard = pn.Column(title , desc, Toronto_realty_dashboard)

## Serve the Panel Dashboard

In [82]:
dashboard.servable()

Column
    [0] Column
        [0] Markdown(str)
    [1] Column
        [0] Markdown(str)
    [2] Tabs
        [0] Column
            [0] Plotly(Figure)
        [1] Column
            [0] HoloViews(Layout)
        [2] Column
            [0] HoloViews(Layout)
        [3] Column
            [0] Column
                [0] Column()
                [1] Row
                    [0] Row(name='interactive09724')
                        [0] HoloViews(DynamicMap, name='interactive09724')
                        [1] Column
                            [0] WidgetBox
                                [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                            [1] VSpacer()
            [1] Column
                [0] Column()
                [1] Row
                    [0] Row(name='interactive09759')
                        [0] HoloViews(DynamicMap, name='interactive09759')
                        [1] Column
                            [0] WidgetBox
                                [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                            [1] VSpacer()
        [4] Row
            [0] Column
                [0] Column()
                [1] Row
                    [0] HoloViews(Bars, name='interactive09803')
            [1] Column
                [0] Column()
                [1] Row
                    [0] Plotly(Figure, name='interactive09886')

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
#neighbourhood_map().show()

In [ ]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [ ]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
# average_house_value()

In [ ]:
# average_value_by_neighbourhood()

In [ ]:
# number_dwelling_types()

In [ ]:
# average_house_value_snapshot()

In [ ]:
# top_most_expensive_neighbourhoods()

In [ ]:
#sunburts_cost_analysis()